# 데이터 살펴보기

In [84]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
from sklearn.model_selection import StratifiedKFold

In [85]:
#파일 불러오기
train = pd.read_csv('data/train.csv', encoding = 'utf-8')
test = pd.read_csv('data/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('data/sample_submission.csv', encoding = 'utf-8')

In [86]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [87]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [88]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


In [89]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


# 전처리

In [90]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [91]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [92]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')

# 모델링

In [107]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [108]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [109]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [110]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d_12  (None, 16)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 24)                408       
_________________________________________________________________
dense_35 (Dense)             (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [111]:
# fit model
num_epochs = 30
history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2, validation_split=0.2)


Epoch 1/30
1372/1372 - 5s - loss: 1.5657 - accuracy: 0.2761 - val_loss: 1.5501 - val_accuracy: 0.2743
Epoch 2/30
1372/1372 - 5s - loss: 1.4760 - accuracy: 0.3333 - val_loss: 1.3920 - val_accuracy: 0.4281
Epoch 3/30
1372/1372 - 5s - loss: 1.2833 - accuracy: 0.4846 - val_loss: 1.2098 - val_accuracy: 0.4929
Epoch 4/30
1372/1372 - 5s - loss: 1.1258 - accuracy: 0.5416 - val_loss: 1.1076 - val_accuracy: 0.5413
Epoch 5/30
1372/1372 - 5s - loss: 1.0366 - accuracy: 0.5807 - val_loss: 1.0508 - val_accuracy: 0.5656
Epoch 6/30
1372/1372 - 5s - loss: 0.9704 - accuracy: 0.6151 - val_loss: 1.0063 - val_accuracy: 0.5940
Epoch 7/30
1372/1372 - 5s - loss: 0.9149 - accuracy: 0.6407 - val_loss: 0.9770 - val_accuracy: 0.6096
Epoch 8/30
1372/1372 - 5s - loss: 0.8662 - accuracy: 0.6634 - val_loss: 0.9519 - val_accuracy: 0.6223
Epoch 9/30
1372/1372 - 5s - loss: 0.8198 - accuracy: 0.6848 - val_loss: 0.9091 - val_accuracy: 0.6455
Epoch 10/30
1372/1372 - 5s - loss: 0.7773 - accuracy: 0.7079 - val_loss: 0.8994 - 

In [112]:
# predict values
pred = model.predict_proba(test_padded)

In [113]:
pred

array([[2.27606593e-04, 7.90963043e-03, 5.92921162e-03, 9.85526800e-01,
        4.06755687e-04],
       [4.54748794e-02, 7.34694183e-01, 1.78611148e-02, 1.16551839e-01,
        8.54180455e-02],
       [9.98364985e-01, 1.63438683e-03, 2.40962397e-08, 6.04079231e-10,
        5.56109967e-07],
       ...,
       [1.01171339e-04, 9.99894261e-01, 1.11166736e-11, 4.58373552e-06,
        2.69483547e-09],
       [1.84388482e-04, 9.99807656e-01, 4.41696818e-10, 7.79635775e-06,
        2.51346904e-07],
       [9.99736011e-01, 2.47497897e-04, 7.68000245e-06, 2.40902889e-08,
        8.84919427e-06]], dtype=float32)

In [100]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,9.790409e-07,6.419500e-02,4.278709e-01,5.079266e-01,6.538886e-06
1,1,3.739731e-03,4.219408e-01,3.318664e-03,5.702551e-01,7.456345e-04
2,2,1.000000e+00,7.744902e-12,5.209777e-12,1.184040e-11,1.744638e-08
3,3,1.252835e-09,9.217658e-17,9.999955e-01,2.006079e-14,4.573841e-06
4,4,9.706652e-01,1.749872e-09,9.807320e-07,2.933384e-02,8.111200e-09
...,...,...,...,...,...,...
19612,19612,1.249102e-08,1.000000e+00,3.733334e-17,2.090645e-11,6.463392e-14
19613,19613,4.483545e-06,2.429229e-12,1.279721e-08,1.557369e-19,9.999955e-01
19614,19614,1.491042e-06,9.999975e-01,2.857540e-10,8.573733e-07,1.589579e-07
19615,19615,1.206705e-05,9.999824e-01,3.441559e-09,2.439870e-07,5.311501e-06


In [101]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')